<a href="https://colab.research.google.com/github/sayyyyed/analisis-sentimen-ikn/blob/main/bielesteem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# read xlsx
df = pd.read_excel('sampel.xlsx')
df

,Nama,Umpan Balik Pengguna
0,Ahmad Arivaldi Machsun,"""Secara umum, website SiBAYAR sudah cukup baik..."
1,Ande,"""Pengalaman saya menggunakan SiBAYAR sangat po..."
2,Zulpi,"""Website SiBAYAR cukup baik dan mudah digunaka..."
3,Cak wer,"""Saya merasa puas dengan pengalaman menggunaka..."
4,Cak to,"""Pengalaman saya menggunakan SiBAYAR sangat ba..."
5,Arivaldi Machsun part 2,"""Website SiBAYAR cukup mudah digunakan, tetapi..."
6,Machsun part 3,"""Saya cukup puas dengan website SiBAYAR, namun..."


In [ ]:
# cleaning data with regex to remove non letter and tanda baca and symbol
df['cleaned'] = df['Umpan Balik Pengguna'].str.replace(r'[^a-zA-Z\s]', '')

In [ ]:
%pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 2.8 MB/s eta 0:00:00


In [ ]:
# stopword removal with sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StopWordRemoverFactory()

stopword = factory.create_stop_word_remover()
df['cleaned'] = df['cleaned'].apply(stopword.remove)

In [ ]:
# stemmer with sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['cleaned'] = df['cleaned'].apply(stemmer.stem)
df.head()

,Nama,Umpan Balik Pengguna,cleaned
0,Ahmad Arivaldi Machsun,"""Secara umum, website SiBAYAR sudah cukup baik...",cara umum website sibayar cukup baik beberapa ...
1,Ande,"""Pengalaman saya menggunakan SiBAYAR sangat po...",alam guna sibayar sangat positif tahap daftar ...
2,Zulpi,"""Website SiBAYAR cukup baik dan mudah digunaka...",website sibayar cukup baik mudah guna beberapa...
3,Cak wer,"""Saya merasa puas dengan pengalaman menggunaka...",saya rasa puas alam guna sibayar beberapa baik...
4,Cak to,"""Pengalaman saya menggunakan SiBAYAR sangat ba...",alam guna sibayar sangat baik umum utama dafta...


In [ ]:
# tokenization to split into words
df['tokenized'] = df['cleaned'].apply(lambda x: x.split())
df.head()


,Nama,Umpan Balik Pengguna,cleaned,tokenized
0,Ahmad Arivaldi Machsun,"""Secara umum, website SiBAYAR sudah cukup baik...",cara umum website sibayar cukup baik beberapa ...,"[cara, umum, website, sibayar, cukup, baik, be..."
1,Ande,"""Pengalaman saya menggunakan SiBAYAR sangat po...",alam guna sibayar sangat positif tahap daftar ...,"[alam, guna, sibayar, sangat, positif, tahap, ..."
2,Zulpi,"""Website SiBAYAR cukup baik dan mudah digunaka...",website sibayar cukup baik mudah guna beberapa...,"[website, sibayar, cukup, baik, mudah, guna, b..."
3,Cak wer,"""Saya merasa puas dengan pengalaman menggunaka...",saya rasa puas alam guna sibayar beberapa baik...,"[saya, rasa, puas, alam, guna, sibayar, bebera..."
4,Cak to,"""Pengalaman saya menggunakan SiBAYAR sangat ba...",alam guna sibayar sangat baik umum utama dafta...,"[alam, guna, sibayar, sangat, baik, umum, utam..."


In [ ]:

def pad_truncate_sequence(seq, maxlen=100, padding_value=0):
    # Truncate if the sequence is longer than maxlen
    if len(seq) > maxlen:
        seq = seq[:maxlen]
    # Pad if the sequence is shorter than maxlen
    elif len(seq) < maxlen:
        padding = [padding_value] * (maxlen - len(seq))
        seq = seq + padding
    return seq

df['tokenized'] = df['tokenized'].apply(lambda x: pad_truncate_sequence(x, maxlen=100))

print(df.head())


                     Nama                               Umpan Balik Pengguna  \
0  Ahmad Arivaldi Machsun  "Secara umum, website SiBAYAR sudah cukup baik...   
1                    Ande  "Pengalaman saya menggunakan SiBAYAR sangat po...   
2                   Zulpi  "Website SiBAYAR cukup baik dan mudah digunaka...   
3                 Cak wer  "Saya merasa puas dengan pengalaman menggunaka...   
4                  Cak to  "Pengalaman saya menggunakan SiBAYAR sangat ba...   

                                             cleaned  \
0  cara umum website sibayar cukup baik beberapa ...   
1  alam guna sibayar sangat positif tahap daftar ...   
2  website sibayar cukup baik mudah guna beberapa...   
3  saya rasa puas alam guna sibayar beberapa baik...   
4  alam guna sibayar sangat baik umum utama dafta...   

                                           tokenized  
0  [cara, umum, website, sibayar, cukup, baik, be...  
1  [alam, guna, sibayar, sangat, positif, tahap, ...  
2  [website, siba

In [ ]:
# word2vec skipgram
from gensim.models import Word2Vec

model = Word2Vec(df['tokenized'], vector_size=100, window=5, min_count=1, workers=4, sg=1)
model.save('word2vec_skipgram.model')

# load model
model = Word2Vec.load('word2vec_skipgram.model')

# get vector of a word
vector = model.wv['website']
print(vector)

[ 0.00227175  0.02505175  0.01123324 -0.0008591   0.0446002  -0.0631377
  0.03482863  0.09624603 -0.07334515 -0.04772593 -0.01791261 -0.07673511
 -0.00171046  0.03213987  0.03305466 -0.01459767  0.00777366 -0.01462059
 -0.00389708 -0.10814035  0.00561448  0.00626097  0.02913334 -0.04284667
 -0.02998636  0.00872063 -0.0513754   0.00978277  0.00731943 -0.00021226
  0.04676311  0.0182746   0.0188066  -0.03660695 -0.03831425  0.08067813
  0.00939508 -0.04380548 -0.05211341 -0.07656457  0.04200746 -0.04686276
 -0.00153441  0.02196928  0.04422552 -0.00456342 -0.04606599 -0.02429504
  0.00990316 -0.00047171  0.03324205 -0.05015938  0.01125317 -0.01320695
 -0.04204001 -0.00249384  0.0062519  -0.01010722 -0.0362127   0.00269116
  0.01099972 -0.01091301  0.06016132  0.00673787 -0.04054834  0.08546146
  0.0378792   0.05630293 -0.09851171  0.0315101  -0.03611321  0.02454563
  0.05174984  0.03562717  0.05210892  0.00484687  0.00565162  0.02554377
 -0.050604    0.00975799 -0.03290013  0.02915598 -0.

In [ ]:
# prompt: biLSTM achitecture, with glorot initialization, relu activation on fully connected layer and softmax in output layer. in training process use categorical class entropy for loss function, backpropagation through time, update weight and parameter BiLSTM, adjust the epoch


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.utils import to_categorical

# ... (Your existing code for data preprocessing) ...

# Assuming 'df' is your DataFrame with preprocessed text and labels
# Replace 'label_column' with the actual column name containing labels
labels = df['label_column'] # Assuming you have a column named 'label_column'

# Convert labels to numerical representation (if necessary)
# ... (Code to convert labels to numerical values if not already numerical) ...

# Convert labels to categorical representation
num_classes = len(np.unique(labels))
y = to_categorical(labels, num_classes=num_classes)


# Create sequences of word vectors
sequences = []
for tokens in df['tokenized']:
    sequence = []
    for token in tokens:
        try:
            sequence.append(model.wv[token])
        except KeyError:
            # Handle out-of-vocabulary words (e.g., use a zero vector)
            sequence.append(np.zeros(100))
    sequences.append(sequence)

X = np.array(sequences)


# BiLSTM model
model = Sequential()
model.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(num_classes, activation='relu', kernel_initializer=GlorotUniform())) # Glorot Initialization, relu activation
model.add(Dense(num_classes, activation='softmax')) # Softmax output layer


# Compile the model
model.compile(loss=CategoricalCrossentropy(), optimizer=Adam(), metrics=['accuracy']) # Categorical Cross-entropy Loss

# Train the model
model.fit(X, y, epochs=10) # Adjust the number of epochs

# ... (Rest of your code for evaluation and further processing)

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec

# Assuming you already have a trained Word2Vec model saved as 'word2vec_skipgram.model'


# Prepare tokenized data (df['tokenized'] should already be tokenized)
# Example of tokenizing the sentences (assuming each element is a list of words)
word_index = {word: idx for idx, word in enumerate(model.wv.index_to_key)}

# Convert tokenized text to sequences of word indices
def text_to_sequences(tokenized_text):
    return [word_index.get(word, 0) for word in tokenized_text]  # 0 for unknown words

X = df['tokenized'].apply(text_to_sequences)

# Assuming you have labels (Y) for classification, e.g., multi-class
# Y should be categorical, and use one-hot encoding if needed
# For this example, we'll assume Y is the label column in your dataframe
Y = to_categorical(df['label'], num_classes=3)  # Example: 3 classes

# Padding sequences to the same length (e.g., 100 words per sequence)
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X, maxlen=100)

# Split data into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_padded, Y, test_size=0.2, random_state=42)

# Build BiLSTM model
model = Sequential()

# Word Embedding Layer - Initialize with pre-trained Word2Vec embeddings
embedding_matrix = np.zeros((len(word2vec_model.wv.index_to_key), word2vec_model.vector_size))

for word, idx in word_index.items():
    if word in word2vec_model.wv:
        embedding_matrix[idx] = word2vec_model.wv[word]

model.add(Embedding(input_dim=len(word2vec_model.wv.index_to_key),
                    output_dim=word2vec_model.vector_size,
                    weights=[embedding_matrix],
                    input_length=100,
                    trainable=False))  # Use pre-trained embeddings and freeze them

# Add Bidirectional LSTM layer
model.add(Bidirectional(LSTM(128, return_sequences=False,
                             kernel_initializer=GlorotUniform(),
                             dropout=0.2)))  # Dropout for regularization

# Fully Connected Layer with ReLU activation
model.add(Dense(64, activation='relu', kernel_initializer=GlorotUniform()))

# Output Layer with Softmax activation for multi-class classification
model.add(Dense(3, activation='softmax'))  # Change 3 to the number of classes in your dataset

# Compile the model
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary of the model
model.summary()

# Train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32,
                    validation_data=(X_val, Y_val),
                    verbose=2)

# Save the model after training
model.save('bilstm_model.h5')



KeyError: 'label'